In [10]:
import findspark
findspark.init()
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import time

In [3]:
sc = SparkSession.builder.master("local[*]").getOrCreate()
spark = SparkSession(sc)

           
                        


In [4]:
movies = spark.read.csv('./movies.csv', header=True, inferSchema = 'true')
ratings = spark.read.csv('./ratings.csv',header = True, inferSchema = 'true')             

In [5]:
movies.show(5)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows



In [6]:
ratings.show(5)

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|    307|   3.5|1256677221|
|     1|    481|   3.5|1256677456|
|     1|   1091|   1.5|1256677471|
|     1|   1257|   4.5|1256677460|
|     1|   1449|   4.5|1256677264|
+------+-------+------+----------+
only showing top 5 rows



In [12]:
#Ratings rating value count ordenados de mayor a menor
start_time = time.time()
ratings.groupBy('rating').count().orderBy('count', ascending=False).show()
print("--- %s seconds ---" % (time.time() - start_time))

+------+-------+
|rating|  count|
+------+-------+
|   4.0|7394710|
|   3.0|5515668|
|   5.0|4071135|
|   3.5|3404360|
|   4.5|2373550|
|   2.0|1850627|
|   2.5|1373419|
|   1.0| 886233|
|   0.5| 442388|
|   1.5| 441354|
+------+-------+

--- 30.28105139732361 seconds ---


In [16]:
start_time = time.time()
means = ratings.groupBy('movieID').agg({'rating':'mean'})
movie_rating = movies.join(means, on = 'movieID', how = 'inner')
movie_rating.show(5)
movie_rating.write.csv('movie_rating.csv')
print("--- %s seconds ---" % (time.time() - start_time))

+-------+--------------------+--------------------+------------------+
|movieId|               title|              genres|       avg(rating)|
+-------+--------------------+--------------------+------------------+
|   1591|        Spawn (1997)|Action|Adventure|...|2.6466656422864165|
|   1645|The Devil's Advoc...|Drama|Mystery|Thr...|3.5352941176470587|
|    471|Hudsucker Proxy, ...|              Comedy| 3.652908677283068|
|   1088|Dirty Dancing (1987)|Drama|Musical|Rom...|3.2480141843971633|
|   1580|Men in Black (a.k...|Action|Comedy|Sci-Fi| 3.578533203874726|
+-------+--------------------+--------------------+------------------+
only showing top 5 rows

--- 31.876930236816406 seconds ---
